In [1]:
import json
import pandas as pd
from openai import OpenAI
from dotenv import dotenv_values
from pycaret.clustering import predict_model, load_model

In [7]:
env = dotenv_values('.env')

openai_client = OpenAI(api_key=env["OPENAI_API_KEY"])

In [9]:
df = pd.read_csv('welcome_survey_simple_v2.csv', sep=';')
df.head()

,age,edu_level,fav_animals,fav_place,gender
0,<18,Podstawowe,Brak ulubionych,NaN,Kobieta
1,25-34,Średnie,Psy,Nad wodą,Mężczyzna
2,45-54,Wyższe,Psy,W lesie,Mężczyzna
3,35-44,Średnie,Koty,W górach,Mężczyzna
4,35-44,Wyższe,Psy,Nad wodą,Mężczyzna


In [12]:
kmeans_pipeline = load_model('welcome_survey_clustering_pipeline_v2')

Transformation Pipeline and Model Successfully Loaded


In [13]:
df_with_clusters = predict_model(model=kmeans_pipeline, data=df)

In [17]:
df_with_clusters["Cluster"].value_counts()

Cluster 0    39
Cluster 6    31
Cluster 1    26
Cluster 8    25
Cluster 2    25
Cluster 3    23
Cluster 4    21
Cluster 7    17
Cluster 5    13
Cluster 9     9
Name: Cluster, dtype: int64

In [24]:
df_with_clusters

,age,edu_level,fav_animals,fav_place,gender,Cluster
0,<18,Podstawowe,Brak ulubionych,NaN,Kobieta,Cluster 5
1,25-34,Średnie,Psy,Nad wodą,Mężczyzna,Cluster 7
2,45-54,Wyższe,Psy,W lesie,Mężczyzna,Cluster 3
3,35-44,Średnie,Koty,W górach,Mężczyzna,Cluster 4
4,35-44,Wyższe,Psy,Nad wodą,Mężczyzna,Cluster 0
...,...,...,...,...,...,...
224,35-44,Wyższe,Koty,Inne,Kobieta,Cluster 2
225,45-54,Wyższe,Inne,W lesie,Mężczyzna,Cluster 3
226,25-34,Wyższe,Psy,W górach,Mężczyzna,Cluster 8
227,35-44,Wyższe,Brak ulubionych,W górach,Mężczyzna,Cluster 2


# Tworzymy prompt od LLM'a

In [18]:
cluster_descriptions = {}
for cluster_id in df_with_clusters['Cluster'].unique():
    cluster_df = df_with_clusters[df_with_clusters['Cluster'] == cluster_id]
    summary = ""
    for column in df_with_clusters:
        if column == 'Cluster':
            continue

        value_counts = cluster_df[column].value_counts()
        value_counts_str = ', '.join([f"{idx}: {cnt}" for idx, cnt in value_counts.items()])
        summary += f"{column} - {value_counts_str}\n"

    cluster_descriptions[cluster_id] = summary 

In [29]:
df_with_clusters["Cluster"].unique()

array(['Cluster 5', 'Cluster 7', 'Cluster 3', 'Cluster 4', 'Cluster 0',
       'Cluster 8', 'Cluster 2', 'Cluster 1', 'Cluster 6', 'Cluster 9'],
      dtype=object)

In [22]:
cluster_df[column].value_counts()

Cluster 9    9
Name: Cluster, dtype: int64

In [23]:
print(cluster_descriptions["Cluster 5"])

age - 35-44: 7, >=65: 2, 45-54: 1, 55-64: 1, <18: 1, unknown: 1, 18-24: 0, 25-34: 0
edu_level - Wyższe: 12, Podstawowe: 1, Średnie: 0
fav_animals - Brak ulubionych: 13, Inne: 0, Koty: 0, Koty i Psy: 0, Psy: 0
fav_place - Nad wodą: 5, W lesie: 4, Inne: 0, W górach: 0
gender - Mężczyzna: 8, Kobieta: 4



In [27]:
prompt = "Użyliśmy algorytmu klastrowania."
for cluster_id, description in cluster_descriptions.items():
    prompt += f"\n\nKlaster {cluster_id}:\n{description}"

prompt += """
Wygeneruj najlepsze nazwy dla każdego z klasterów oraz ich opisy

Użyj formatu JSON. Przykładowo:
{
    "Cluster 0": {
        "name": "Klaster 0",
        "description": "W tym klastrze znajdują się osoby, które..."
    },
    "Cluster 1": {
        "name": "Klaster 1",
        "description": "W tym klastrze znajdują się osoby, które..."
    }
}
"""
print(prompt)

Użyliśmy algorytmu klastrowania.

Klaster Cluster 5:
age - 35-44: 7, >=65: 2, 45-54: 1, 55-64: 1, <18: 1, unknown: 1, 18-24: 0, 25-34: 0
edu_level - Wyższe: 12, Podstawowe: 1, Średnie: 0
fav_animals - Brak ulubionych: 13, Inne: 0, Koty: 0, Koty i Psy: 0, Psy: 0
fav_place - Nad wodą: 5, W lesie: 4, Inne: 0, W górach: 0
gender - Mężczyzna: 8, Kobieta: 4


Klaster Cluster 7:
age - 35-44: 8, 45-54: 4, 18-24: 2, 25-34: 2, >=65: 1, 55-64: 0, <18: 0, unknown: 0
edu_level - Średnie: 17, Podstawowe: 0, Wyższe: 0
fav_animals - Psy: 13, Koty: 2, Brak ulubionych: 1, Inne: 1, Koty i Psy: 0
fav_place - Nad wodą: 12, W lesie: 2, Inne: 0, W górach: 0
gender - Mężczyzna: 12, Kobieta: 5


Klaster Cluster 3:
age - 45-54: 20, 55-64: 3, 18-24: 0, 25-34: 0, 35-44: 0, <18: 0, >=65: 0, unknown: 0
edu_level - Wyższe: 23, Podstawowe: 0, Średnie: 0
fav_animals - Psy: 13, Koty: 5, Inne: 4, Brak ulubionych: 1, Koty i Psy: 0
fav_place - W lesie: 21, Inne: 2, Nad wodą: 0, W górach: 0
gender - Mężczyzna: 19, Kobieta:

In [32]:
response = openai_client.chat.completions.create(
    model="gpt-4o",
    temperature=0,
    messages=[
        {
            "role": "user",
            "content": [{"type": "text", "text": prompt}],
        }
    ],
)

In [33]:
response

ChatCompletion(id='chatcmpl-D62DNGoL6v4Kybe1idTjnMQeH2Ghw', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{\n    "Cluster 0": {\n        "name": "Miłośnicy Wody z Wyższym Wykształceniem",\n        "description": "W tym klastrze znajdują się osoby w wieku 35-44 lat, z wyższym wykształceniem, które preferują spędzanie czasu nad wodą. Większość z nich to mężczyźni, a ich ulubionymi zwierzętami są psy."\n    },\n    "Cluster 1": {\n        "name": "Górscy Profesjonaliści",\n        "description": "W tym klastrze znajdują się osoby w wieku 45-54 lat, z wyższym wykształceniem, które preferują spędzanie czasu w górach. Większość z nich to mężczyźni, a ich ulubionymi zwierzętami są psy."\n    },\n    "Cluster 2": {\n        "name": "Kociarze Górscy",\n        "description": "W tym klastrze znajdują się osoby w wieku 35-44 lat, z wyższym wykształceniem, które preferują spędzanie czasu w górach. Większość z nich to mężczyźni, a ich 

In [34]:
result = response.choices[0].message.content.replace("```json", "").replace("```", "").strip()
cluster_names_and_descriptions = json.loads(result)

In [35]:
with open ("welcome_survey_v2.json", "w") as f:
    f.write(json.dumps(cluster_names_and_descriptions))

In [37]:
with open ("welcome_survey_v2.json", "r") as f:
    print(json.loads(f.read()))

{'Cluster 0': {'name': 'Miłośnicy Wody z Wyższym Wykształceniem', 'description': 'W tym klastrze znajdują się osoby w wieku 35-44 lat, z wyższym wykształceniem, które preferują spędzanie czasu nad wodą. Większość z nich to mężczyźni, a ich ulubionymi zwierzętami są psy.'}, 'Cluster 1': {'name': 'Górscy Profesjonaliści', 'description': 'W tym klastrze znajdują się osoby w wieku 45-54 lat, z wyższym wykształceniem, które preferują spędzanie czasu w górach. Większość z nich to mężczyźni, a ich ulubionymi zwierzętami są psy.'}, 'Cluster 2': {'name': 'Kociarze Górscy', 'description': 'W tym klastrze znajdują się osoby w wieku 35-44 lat, z wyższym wykształceniem, które preferują spędzanie czasu w górach. Większość z nich to mężczyźni, a ich ulubionymi zwierzętami są koty.'}, 'Cluster 3': {'name': 'Leśni Profesjonaliści', 'description': 'W tym klastrze znajdują się osoby w wieku 45-54 lat, z wyższym wykształceniem, które preferują spędzanie czasu w lesie. Większość z nich to mężczyźni, a ich 

In [51]:
with open ("welcome_survey_v2.json", "r") as f:
    #example = f.read()
    #print(json.loads(example))
    example = json.load(f)
    print(example)

{'Cluster 0': {'name': 'Miłośnicy Wody z Wyższym Wykształceniem', 'description': 'W tym klastrze znajdują się osoby w wieku 35-44 lat, z wyższym wykształceniem, które preferują spędzanie czasu nad wodą. Większość z nich to mężczyźni, a ich ulubionymi zwierzętami są psy.'}, 'Cluster 1': {'name': 'Górscy Profesjonaliści', 'description': 'W tym klastrze znajdują się osoby w wieku 45-54 lat, z wyższym wykształceniem, które preferują spędzanie czasu w górach. Większość z nich to mężczyźni, a ich ulubionymi zwierzętami są psy.'}, 'Cluster 2': {'name': 'Kociarze Górscy', 'description': 'W tym klastrze znajdują się osoby w wieku 35-44 lat, z wyższym wykształceniem, które preferują spędzanie czasu w górach. Większość z nich to mężczyźni, a ich ulubionymi zwierzętami są koty.'}, 'Cluster 3': {'name': 'Leśni Profesjonaliści', 'description': 'W tym klastrze znajdują się osoby w wieku 45-54 lat, z wyższym wykształceniem, które preferują spędzanie czasu w lesie. Większość z nich to mężczyźni, a ich 